# COLLECTING DATA: API

#### Import required libraries

In [38]:
import requests
import pandas as pd
import numpy as np
import datetime

#### Initiate a request object

In [39]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'
response = requests.get(static_json_url)

#### Functions to extract data from the html table

In [40]:
# Setting this option will print all collumns of a dataframe
pd.set_option("display.max_columns", None)
# Setting this option will print all of the data in a feature
pd.set_option("display.max_colwidth", None)

In [41]:
# This function uses the rocket column to call the API and append the extracted information.
def getBoosterVersion(data):
    for x in data["rocket"]:
        response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(x)).json()
        BoosterVersion.append(response["name"])

# This function uses the launchpad column to call the API and append the extracted information.
def getLaunchSite(data):
    for x in data["launchpad"]:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/" + str(x)).json()
            Longitude.append(response["longitude"])
            Latitude.append(response["latitude"])
            LaunchSite.append(response["name"])


# This function uses the payloads column to call the API and append the extracted infromation.
def getPayloadData(data):
    for load in data["payloads"]:
        if load:
            response = requests.get("https://api.spacexdata.com/v4/payloads/" + load).json()
            PayloadMass.append(response["mass_kg"])
            Orbit.append(response["orbit"])

# This function uses the cores column to call the API and append the extracted data. 
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

#### Creating a dataframe for making API calls

In [42]:
# Use json_normalize meethod to convert the json result into a dataframe
data = pd.json_normalize(response.json())

In [43]:
# Create a subset of desired features + the flight number and date_utc.
data = data[["rocket", "payloads", "launchpad", "cores", "flight_number", "date_utc"]]

# Remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters 
# and rows that have multiple payloads in a single rocket.
data = data[data["cores"].map(len) == 1]
data = data[data["payloads"].map(len) == 1]

# Extract the single value in the list and replace the feature.
data["cores"] = data["cores"].map(lambda x : x[0])
data["payloads"] = data["payloads"].map(lambda x : x[0])

# Convert the date_utc to a datetime datatype and then extract the date dropping the time element. 
data["date"] = pd.to_datetime(data["date_utc"]).dt.date

# Restrict the dates of the launches before 11/13/2020
data = data[data["date"] <= datetime.date(2020, 11, 13)]

In [44]:
data.head()

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04


In [45]:
#Global variables 
# Empty lists to store data from the API requests
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [46]:
#### Initiate calls to the API via the functions. 

In [47]:
# Call getBoosterVersion
getBoosterVersion(data)

# Call getLaunchSite
getLaunchSite(data)

# Call getPayloadData
getPayloadData(data)

# Call getCoreData
getCoreData(data)

#### Create a final dataframe with the extracted data.

In [48]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

In [49]:
# Convert launch_dict to a dataframe. 
launch_df = pd.DataFrame.from_dict(launch_dict, orient = "index")
final_df = launch_df.transpose()

In [50]:
final_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,None,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,None,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0003,-80.577366,28.561857


In [51]:
# Filter to Falcon 9 launches only
final_falcon9_df = final_df[final_df["BoosterVersion"] != "Falcon 1"]
final_falcon9_df["BoosterVersion"].unique()

array(['Falcon 9'], dtype=object)

In [52]:
# Reset the FlightNumber column from 1. 
final_falcon9_df.loc[:,'FlightNumber'] = list(range(1, final_falcon9_df.shape[0]+1))
print(len(final_falcon9_df["FlightNumber"]))
final_falcon9_df.head()

90


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,None,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B1004,-80.577366,28.561857


In [53]:
# Check for missing values
final_falcon9_df.isna().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [54]:
# Calculate the mean for PayloadMass
avg_payloadmass = final_falcon9_df["PayloadMass"].astype("float").mean(axis=0)
# Replace nan with the mean
final_falcon9_df["PayloadMass"].replace(np.nan, avg_payloadmass, inplace = True)
final_falcon9_df.isnull().sum()

C:\Users\OK\AppData\Local\Temp\ipykernel_15604\946798594.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_falcon9_df["PayloadMass"].replace(np.nan, avg_payloadmass, inplace = True)


FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64